# IMPORTS

In [ ]:
from collections import *
from functools import *
from itertools import *
import operator
import sys
import re
import math
import string
import bisect
import parse
import matplotlib.pyplot as plt
import numpy as np
import heapq
from matplotlib.path import Path
import ast
import hashlib
import unittest
import copy
from dataclasses import dataclass, field
import sympy
import networkx as nx
from pyvis.network import Network
from typing import *
from shapely.geometry import Polygon
from enum import StrEnum, Enum
from abc import ABC, abstractmethod

In [105]:
DIRECTION = {
    "<": (0, -1),
    ">": (0, 1),
    "^": (-1, 0),
    "v": (1, 0)
}
MAP = {
    ".": "..",
    "@": "@.",
    "#": "##",
    "O": "[]"
}
def score(r, c):
    return 100 * r + c
def calculate(filename):
    def display_data(grid):
        return "\n".join(["".join(row) for row in grid])
    def in_bounds(r, c):
        return 0 <= r < R and 0 <= c < C
    def get_boxes(r, c, dr, dc):
        q = deque([(r, c)])
        boxes = set()
        while q:
            r, c = q.popleft()
            nr, nc = r + dr, c + dc
            if not in_bounds(nr, nc): continue
            if grid[nr][nc] == "#": return False, []
            if (nr, nc) in point_to_box:
                b = point_to_box[(nr, nc)]
                if b not in boxes:
                    boxes.add(b)
                    for rr, cc in box_to_point[b]:
                        q.append((rr, cc))
        return True, boxes

    with open(filename, "r") as f:     
        instructions = []
        grid = []
        flag = False
        for line in f.read().splitlines():
            if not line:
                flag = True
                continue
            if flag:
                instructions.extend(line)
            else:
                row = "".join(map(lambda x: MAP[x], line))
                grid.append(list(row))
        R, C = len(grid), len(grid[0])
        for r, c in product(range(R), range(C)):
            if grid[r][c] == "@":
                break
        box_to_point = defaultdict(list)
        point_to_box = {}
        for r, c in product(range(R), range(C)):
            if grid[r][c] == "]":
                b = len(box_to_point)
                box_to_point[b].extend([(r, c), (r, c - 1)])
                point_to_box[(r, c)] = point_to_box[(r, c - 1)] = b
        for d in instructions:
            dr, dc = DIRECTION[d]
            can_move, boxes = get_boxes(r, c, dr, dc)
            if not can_move: continue
            for b in boxes:
                points = copy.deepcopy(box_to_point[b])
                box_to_point[b].clear()
                # update points
                for r, c in points:
                    box_to_point[b].append((r + dr, c + dc))
                    point_to_box[(r + dr, c + dc)] = b
                    if point_to_box[(r, c)] == b:
                        del point_to_box[(r, c)]
            r += dr 
            c += dc
        ans = 0
        for b in range(len(box_to_point)):
            r1, c1 = box_to_point[b][0]
            r2, c2 = box_to_point[b][1]
            ans += score(r1, min(c1, c2))
        print(ans)

calculate("small.txt")
# calculate("big.txt")

9802


In [104]:
dic = {

}

dic.setdefault(2)
print(dic)

{2: None}


In [149]:
from PIL import Image, ImageDraw, ImageFont
def resize_image(input_path, output_path, scale_factor=0.5):
    """
    Resize the image by a scale factor to reduce its height and width proportionally.

    :param input_path: Path to the input image.
    :param output_path: Path to save the resized image.
    :param scale_factor: Scale factor to resize the image (e.g., 0.5 to reduce by 50%).
    """
    image = Image.open(input_path)
    new_width = int(image.width * scale_factor)
    new_height = int(image.height * scale_factor)
    resized_image = image.resize((new_width, new_height), Image.LANCZOS)
    resized_image.save(output_path, format="PNG")
    
def save_ascii_art_to_png(ascii_art, output_path, font_path=None, font_size=12, image_bg_color="black", text_color="white"):
    """
    Save ASCII art as a PNG image.

    :param ascii_art: The ASCII art string.
    :param output_path: The path where the PNG will be saved.
    :param font_path: Path to the .ttf font file (optional, defaults to system font).
    :param font_size: Font size for rendering the text.
    :param image_bg_color: Background color of the image.
    :param text_color: Color of the ASCII text.
    """
    # Split ASCII art into lines
    lines = ascii_art.splitlines()
    max_line_length = max(len(line) for line in lines)
    num_lines = len(lines)

    # Load a font (use default if font_path is None)
    if font_path is None:
        font = ImageFont.load_default()
    else:
        font = ImageFont.truetype(font_path, font_size)



    # Calculate image dimensions using textbbox()
    dummy_image = Image.new("RGB", (1, 1))
    draw = ImageDraw.Draw(dummy_image)
    char_width = int(draw.textlength("A", font=font))  # Width of one character
    char_height = int(font.getbbox("A")[3])           # Height of one character
    line_spacing = 1
    # Adjust line height for compactness
    line_height = int(char_height * line_spacing)
    image_width = int(char_width * max_line_length)
    image_width = 80
    image_height = int(line_height * num_lines)

    # Create the image
    image = Image.new("RGB", (image_width, image_height), color=image_bg_color)
    draw = ImageDraw.Draw(image)

    # Draw the ASCII art onto the image
    for i, line in enumerate(lines):
        draw.text((0, i * char_height), line, fill=text_color, font=font)

    # Save the image as PNG
    image.save(output_path, format="PNG")
    print(f"ASCII art saved to {output_path}")
def calculate(filename):
    def get_grid(data):
        grid = [[" " for _ in range(C)] for _ in range(R)]
        for r, c in data:
            grid[r][c] = "#"
        return grid
    def display_data(grid):
        return "\n".join(["".join(row) for row in grid])
    def extract(line):
        x, y, w, z = map(int, re.findall(pat, line))
        return x, y, w, z
    def row_count(grid, threshold):
        cnt = 0
        for r, c in product(range(R), range(C)):
            if r == 0 or grid[r][c] != "#": cnt = 0
            if grid[r][c] == "#": cnt += 1
            if cnt > threshold: return True
        return False
    with open(filename, "r") as f:
        data = f.read().splitlines()
        pat = r"(-?\d+)"
        ans = 1
        R, C = 103, 101
        N = len(data)
        print("N", N)
        robots = []
        speed = []
        for line in data:
            c, r, dc, dr = extract(line)
            robots.append([r, c])
            speed.append((dr, dc))
        
        for i in range(10_000):
            grid = get_grid(robots)
            if row_count(grid, 5):
                ascii_art = display_data(grid)
                path = f"images/{i}.png"
                save_ascii_art_to_png(ascii_art, path, font_path = "/usr/share/fonts/truetype/dejavu/DejaVuSerif.ttf", font_size = 2)
            for i in range(N):
                robots[i][0] += speed[i][0]
                robots[i][0] %= R
                robots[i][1] += speed[i][1]
                robots[i][1] %= C


        print(ans)
# calculate("small.txt")
calculate("big.txt")

N 500
ASCII art saved to images/23.png
ASCII art saved to images/3916.png
ASCII art saved to images/5461.png
ASCII art saved to images/6285.png
ASCII art saved to images/7006.png
ASCII art saved to images/7497.png
1


In [107]:
from PIL import Image, ImageDraw, ImageFont

def save_ascii_art_to_png(ascii_art, output_path, font_path=None, font_size=12, image_bg_color="black", text_color="white"):
    """
    Save ASCII art as a PNG image.

    :param ascii_art: The ASCII art string.
    :param output_path: The path where the PNG will be saved.
    :param font_path: Path to the .ttf font file (optional, defaults to system font).
    :param font_size: Font size for rendering the text.
    :param image_bg_color: Background color of the image.
    :param text_color: Color of the ASCII text.
    """
    # Split ASCII art into lines
    lines = ascii_art.splitlines()
    max_line_length = max(len(line) for line in lines)
    num_lines = len(lines)

    # Load a font (use default if font_path is None)
    if font_path is None:
        font = ImageFont.load_default()
    else:
        font = ImageFont.truetype(font_path, font_size)

    # Calculate image dimensions using textbbox()
    dummy_image = Image.new("RGB", (1, 1))
    draw = ImageDraw.Draw(dummy_image)
    char_width = int(draw.textlength("A", font=font))  # Width of one character
    char_height = int(font.getbbox("A")[3])           # Height of one character
    image_width = int(char_width * max_line_length)
    image_height = int(char_height * num_lines)

    # Create the image
    image = Image.new("RGB", (image_width, image_height), color=image_bg_color)
    draw = ImageDraw.Draw(image)

    # Draw the ASCII art onto the image
    for i, line in enumerate(lines):
        draw.text((0, i * char_height), line, fill=text_color, font=font)

    # Save the image as PNG
    image.save(output_path, format="PNG")
    print(f"ASCII art saved to {output_path}")

# Example ASCII art
ascii_art = """
  #####
 #     #
 #     #
  #####
       #
       #
"""

# Save ASCII art to a PNG file
save_ascii_art_to_png(ascii_art, "ascii_art.png", font_size=14)


ASCII art saved to ascii_art.png


In [102]:
6 + 4 + 2
res = 0
i = 1
cnt = 0
while res < 500:
    res += i
    cnt += 1
    print(cnt, i, res)
    i += 2
print(res)

1 1 1
2 3 4
3 5 9
4 7 16
5 9 25
6 11 36
7 13 49
8 15 64
9 17 81
10 19 100
11 21 121
12 23 144
13 25 169
14 27 196
15 29 225
16 31 256
17 33 289
18 35 324
19 37 361
20 39 400
21 41 441
22 43 484
23 45 529
529


In [100]:
1 + 3 + 5 + 7

16

In [ ]:
def calculate(filename):
    def is_safe()
        pass
    def is_set(i, mask):
        return (mask >> i) & 1
    with open(filename, "r") as f:
        N = 4
        end_mask = (1 << N) - 1
        state = [0] * 4
        for i in [0, 2]:
            state[0] += 1 << i
        state[1] += 1 << 1
        state[2] += 1 << 3
        vis = set([tuple(state)])
        q = deque(tuple(state))
        steps = 0
        while q:
            n = len(q)
            for _ in range(q):
                state = q.popleft()
                if state[-1] == end_mask: 
                    print(steps)
                    return
                for lv in range(4):
                    items = []
                    for i in range(N):
                        if is_set(i, state[lv]): items.append(i)
                    for item in items:
                        if is_valid(item, lv + 1):
                            nstate = copy.deepcopy(state)
                        if is_valid(item, lv - 1):


            steps += 1
calculate("small.txt")
calculate("big.txt")

{(5, 2, 8, 0)}
[5, 2, 8, 0]
{(5, 2, 8, 0)}
[5, 2, 8, 0]


In [ ]:
from parse import compile

def calculate(filename):
    with open(filename, "r") as f:
        data = f.read().splitlines()
        row_translate = compile("rotate row y={:d} by {:d}")
        column_translate = compile("rotate column x={:d} by {:d}")
        rect = compile("rect {:d}x{:d}")
        ans = 0
        for line in data:
            print(line)
        print(ans)

# calculate("small.txt")
calculate("big.txt")

rect 1x1
rotate row y=0 by 20
rect 1x1
rotate row y=0 by 2
rect 1x1
rotate row y=0 by 3
rect 2x1
rotate row y=0 by 2
rect 1x1
rotate row y=0 by 3
rect 2x1
rotate row y=0 by 2
rect 1x1
rotate row y=0 by 4
rect 2x1
rotate row y=0 by 2
rect 1x1
rotate row y=0 by 2
rect 1x1
rotate row y=0 by 2
rect 1x1
rotate row y=0 by 3
rect 2x1
rotate row y=0 by 2
rect 1x1
rotate row y=0 by 5
rect 1x1
rotate row y=0 by 2
rect 1x1
rotate row y=0 by 6
rect 5x1
rotate row y=0 by 2
rect 1x3
rotate row y=2 by 8
rotate row y=0 by 8
rotate column x=0 by 1
rect 7x1
rotate row y=2 by 24
rotate row y=0 by 20
rotate column x=5 by 1
rotate column x=4 by 2
rotate column x=2 by 2
rotate column x=0 by 1
rect 7x1
rotate column x=34 by 2
rotate column x=22 by 1
rotate column x=15 by 1
rotate row y=2 by 18
rotate row y=0 by 12
rotate column x=8 by 2
rotate column x=7 by 1
rotate column x=5 by 2
rotate column x=2 by 1
rotate column x=0 by 1
rect 9x1
rotate row y=3 by 28
rotate row y=1 by 28
rotate row y=0 by 20
rotate col

In [ ]:
34, 1150